In [1]:
from google.colab import drive
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

drive.mount('/content/drive', force_remount=True)
df = pd.read_excel("/content/drive/MyDrive/AI/dataset/Energy_Power_Weather.xlsx", sheet_name=1)

Mounted at /content/drive


# 1.Tiền xử lý dữ liệu

In [2]:
# Chuyển cột 'Timestamp' sang kiểu datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Lấy chỉ thời gian và định dạng thành chuỗi '00:00:00'
df['Time'] = df['Timestamp'].dt.strftime('%H:%M:%S')

df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.time

# Tạo cột mới cho giờ
df['Hour'] = df['Timestamp'].dt.hour

df['Date'] = df['Timestamp'].dt.date

df['YearMonth'] = df['Timestamp'].dt.to_period('M').astype(str)

df

,Timestamp,Watt,TempMin,TempMax,Weather,Time,Hour,Date,YearMonth
0,2019-01-28 00:00:00,0.0,18,23,Patchy rain possible,00:00:00,0,2019-01-28,2019-01
1,2019-01-28 00:15:00,0.0,18,23,Patchy rain possible,00:15:00,0,2019-01-28,2019-01
2,2019-01-28 00:30:00,0.0,18,23,Patchy rain possible,00:30:00,0,2019-01-28,2019-01
3,2019-01-28 00:45:00,0.0,18,23,Patchy rain possible,00:45:00,0,2019-01-28,2019-01
4,2019-01-28 01:00:00,0.0,18,23,Patchy rain possible,01:00:00,1,2019-01-28,2019-01
...,...,...,...,...,...,...,...,...,...
168283,2023-11-25 22:45:00,0.0,21,24,Light rain shower,22:45:00,22,2023-11-25,2023-11
168284,2023-11-25 23:00:00,0.0,21,24,Light rain shower,23:00:00,23,2023-11-25,2023-11
168285,2023-11-25 23:15:00,0.0,21,24,Light rain shower,23:15:00,23,2023-11-25,2023-11
168286,2023-11-25 23:30:00,0.0,21,24,Light rain shower,23:30:00,23,2023-11-25,2023-11


In [3]:
df_tem = df.groupby(["Date", "Weather"]).agg({'TempMin': 'mean', 'TempMax': 'mean', 'Watt': 'sum'}).reset_index()
#df_tem = df_tem.drop(columns=["Date"], axis=1)
df_tem

,Date,Weather,TempMin,TempMax,Watt
0,2019-01-28,Patchy rain possible,18.0,23.0,8972.480
1,2019-01-29,Patchy rain possible,20.0,25.0,25845.085
2,2019-01-30,Patchy rain possible,21.0,24.0,36149.643
3,2019-01-31,Sunny,21.0,25.0,65131.089
4,2019-02-01,Moderate rain at times,19.0,23.0,36141.824
...,...,...,...,...,...
1748,2023-11-21,Patchy rain possible,22.0,26.0,45215.285
1749,2023-11-22,Light rain shower,20.0,25.0,19460.444
1750,2023-11-23,Light rain shower,23.0,25.0,56680.677
1751,2023-11-24,Patchy rain possible,22.0,26.0,56748.063


In [4]:
import pandas as pd


# Tạo dãy ngày từ ngày bắt đầu đến ngày kết thúc
all_dates = pd.date_range(start='2019-01-28', end='2023-11-25')

# Tìm những ngày thiếu sót
missing_dates = all_dates.difference(df_tem['Date'])

# In ra những ngày thiếu sót
print(missing_dates)


DatetimeIndex(['2019-04-27', '2020-06-05', '2020-09-18', '2020-10-28',
               '2021-01-20', '2021-01-21', '2021-01-22', '2021-01-23',
               '2022-09-28', '2022-09-29'],
              dtype='datetime64[ns]', freq=None)


In [22]:
weather = {'Patchy rain possible': 0.27,
           'Sunny': 0.99,
           'Moderate rain at times': 0.22,
           'Overcast': 0.5,
           'Partly cloudy': 0.67,
           'Cloudy': 0.56,
           'Heavy rain at times': 0.16,
           'Moderate or heavy rain shower': 0.11,
           'Light rain shower': 0.33,
           'Mist': 0.44,
           'Patchy light rain with thunder': 0.24,
           'Thundery outbreaks possible': 0.31,
           'Patchy light drizzle': 0.39,
           'Torrential rain shower': 0.01
}

df_tem['Weather'] = df_tem['Weather'].map(weather)
df_tem["Watt"] = df_tem["Watt"] / 1000
df_tem

,Weather,TempMin,TempMax,Watt
0,0.27,18.0,23.0,8.972480
1,0.27,20.0,25.0,25.845085
2,0.27,21.0,24.0,36.149643
3,0.99,21.0,25.0,65.131089
4,0.22,19.0,23.0,36.141824
...,...,...,...,...
1748,0.27,22.0,26.0,45.215285
1749,0.33,20.0,25.0,19.460444
1750,0.33,23.0,25.0,56.680677
1751,0.27,22.0,26.0,56.748063


# 2.Tạo dữ liệu

In [23]:
from sklearn.model_selection import train_test_split

X = []
y = []

n = 14
m = 7

dataframe = df_tem.values
for i in range(len(dataframe) - (m + n)):
    X.append(dataframe[i:i+n])
    y.append(dataframe[i+n:i+n+m, -1])
X = np.array(X)
y = np.array(y)

print(X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)

(1732, 14, 4) (1732, 7)
(1385, 14, 4)


# 3.Huấn luyện

### 3.1. LSTM

In [ ]:
from keras import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.metrics import RootMeanSquaredError


def build_lstm(X_train, y_train):
    n_samples, timesteps, n_feature = X_train.shape
    model = Sequential()
    model.add(LSTM(200, activation="tanh", input_shape=(timesteps, n_feature)))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1))
    model.compile(loss="mse", optimizer='adam', metrics=[RootMeanSquaredError(), "mae"])
    model.fit(X_train, y_train, epochs=100, batch_size=16, verbose=0)
    return model

md = build_lstm(X_train, y_train)

In [ ]:
print(md.evaluate(X_test, y_test, verbose=0))
print(md.predict(X_test, verbose=0)[:7])
print(y_test[0])

[365513312.0, 19118.40234375, 15310.7294921875]
[[56264.156]
 [42883.094]
 [32466.273]
 [51831.39 ]
 [61256.52 ]
 [55838.824]
 [61256.676]]
[41383.527 13233.268  2603.151  1839.83   2294.786  6128.297  3675.964]


### 3.2 CNN-LSTM

In [25]:
from keras.layers import Conv1D
from keras.layers import MaxPool1D
from keras.layers import Flatten
from keras.layers import RepeatVector
from keras import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.metrics import RootMeanSquaredError
from keras.layers import Dropout

def build_cnn_lstm(X_train, y_train):
    n_samples, time_step, n_feature = X_train.shape
    md = Sequential()
    md.add(Conv1D(64, kernel_size=3, activation="relu", input_shape=(time_step, n_feature)))
    md.add(MaxPool1D(2))
    md.add(Conv1D(128, kernel_size=3, activation="relu"))
    md.add(MaxPool1D(2))
    md.add(Dropout(0.2))
    md.add(Flatten())
    md.add(RepeatVector(time_step))
    md.add(LSTM(200, activation="tanh"))
    md.add(Dropout(0.2))
    md.add(Dense(100, activation='relu'))
    md.add(Dense(7))
    md.compile(loss="mse", optimizer="adam", metrics=[RootMeanSquaredError(), "mae"])
    md.fit(X_train, y_train, epochs=100, batch_size=16, verbose=0)
    return md

cnn_lstm = build_cnn_lstm(X_train, y_train)
print(cnn_lstm.evaluate(X_test, y_test, verbose=0))
print(cnn_lstm.predict(X_test, verbose=0)[:7])
print(y_test[0])

[283.5793762207031, 16.839815139770508, 12.600238800048828]
[[ 1.2467093  7.3499136  9.439655  11.862197   9.917661   8.949959
   7.1752515]
 [38.445026  45.13749   52.083954  49.673744  41.944927  34.34969
  38.59533  ]
 [26.515717  30.302     32.44821   26.84111   21.906298  16.552025
  17.047272 ]
 [48.594543  50.05744   51.37903   52.94491   51.947426  49.66129
  48.613018 ]
 [70.474174  67.71916   66.45283   66.221825  64.18984   69.38136
  71.63671  ]
 [18.475483   9.866447   7.7642436 17.677301  28.571651  47.154976
  60.799343 ]
 [68.862305  64.71688   64.80293   63.63835   60.064743  61.7919
  60.77134  ]]
[41.383527 13.233268  2.603151  1.83983   2.294786  6.128297  3.675964]


In [26]:
cnn_lstm.save('cnn_lstm.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## 3.3 Conv LSTM custom


In [24]:
from keras import Model
from keras.layers import Input, Dense, MaxPooling1D, Conv1D, LSTM, Flatten, RepeatVector
from keras.layers import Dropout, Add, concatenate
from keras.utils import plot_model


def build_custom_model(X_train, y_train):
    n_samples, time_step, n_feature = X_train[0].shape
    input_1 = Input(shape=(time_step, n_feature))
    x1 = Conv1D(filters=64, kernel_size=3, activation='relu')(input_1)
    x1 = MaxPooling1D(pool_size=2)(x1)
    x1 = Conv1D(filters=128, kernel_size=3, activation='relu')(x1)
    x1 = MaxPooling1D(pool_size=2)(x1)
    x1 = Dropout(0.2)(x1)
    x1 = Flatten()(x1)
    x1 = RepeatVector(time_step)(x1)
    x1 = LSTM(200, activation='tanh')(x1)
    x1 = Dropout(0.2)(x1)
    x1 = Dense(100, activation='relu')(x1)
    x1 = Dense(1)(x1)

    input_2 = Input(shape=(time_step, 1))
    x2 = LSTM(200, activation='tanh')(input_2)
    x2 = Dropout(0.2)(x2)
    x2 = Dense(100, activation='relu')(x2)
    x2 = Dense(1)(x2)

    x = concatenate([x1, x2])
    x = Dense(100, activation='relu')(x)
    x = Dropout(0.2)(x)
    output = Dense(7)(x)

    md = Model(inputs=[input_1, input_2], outputs=output)
    md.compile(loss="mse", optimizer="adam", metrics=["mae"])
    md.fit(X_train, y_train, epochs=10, batch_size=16, verbose=1)
    return md

train_1 = X_train[:,:,:-1]
train_2 = X_train[:,:, -1]
md = build_custom_model([train_1, train_2], y_train)

Epoch 1/10
87/87 [==============================] - 8s 10ms/step - loss: 1317.3638 - mae: 29.4185
Epoch 2/10
87/87 [==============================] - 1s 12ms/step - loss: 425.6074 - mae: 16.5746
Epoch 3/10
87/87 [==============================] - 1s 12ms/step - loss: 380.8205 - mae: 15.4085
Epoch 4/10
87/87 [==============================] - 1s 12ms/step - loss: 379.8051 - mae: 15.4003
Epoch 5/10
87/87 [==============================] - 1s 12ms/step - loss: 378.0163 - mae: 15.3107
Epoch 6/10
87/87 [==============================] - 1s 8ms/step - loss: 369.2372 - mae: 15.1755
Epoch 7/10
87/87 [==============================] - 1s 8ms/step - loss: 367.1940 - mae: 15.0778
Epoch 8/10
87/87 [==============================] - 1s 8ms/step - loss: 366.6680 - mae: 15.0163
Epoch 9/10
87/87 [==============================] - 1s 8ms/step - loss: 355.0409 - mae: 14.8682
Epoch 10/10
87/87 [==============================] - 1s 8ms/step - loss: 360.7385 - mae: 14.9693


In [43]:
test_1 = X_test[:,:,:-1]
test_2 = X_test[:,:, -1]
print(md.evaluate([test_1, test_2], y_test, verbose=0))
print(md.predict([test_1, test_2], verbose=0)[:7])
print(y_test[0])

[338.29302978515625, 14.93432331085205]
[[27.247032]
 [46.30484 ]
 [64.57392 ]
 [60.77286 ]
 [46.20261 ]
 [26.348846]
 [36.33568 ]]
[19.272358 11.93189  13.724976 41.46099  15.496222 49.91443  37.246283]
